# Modelling 1

### Library and data imports

In [1]:
from importlib import reload
import pandas as pd
import numpy as np
from spektral.data import Dataset, Graph
import dataset
reload(dataset)
import pickle
import os

In [2]:
# Load combined data
df_raw = pd.read_csv('../data/combined.csv')
df = df_raw.copy()

# Load hero feature data
df_features = pd.read_csv('../data/features.csv')
df_features = df_features.set_index('hero_id')

# Load standard filter
df_filter = pd.read_csv('../models/filter.csv')

In [3]:
# Load graph dataset 50000 matches at a time
dir = '../data/graphs_v1/'
count = 0
total = len(df)
step = 50000

for i in range(0,int(np.ceil(total/step))):
    start = i*step
    end = start+step-1 if (start+step)<total else total-1
    path = dir+f'graphs_v1_{start}-{end}.pkl'
    print(path)
    file = open(path,'rb')
    if i==0:
        graphs = pickle.load(file)
    else:
        graphs = graphs + pickle.load(file)

../data/graphs_v1/graphs_v1_0-49999.pkl
../data/graphs_v1/graphs_v1_50000-99999.pkl
../data/graphs_v1/graphs_v1_100000-149999.pkl
../data/graphs_v1/graphs_v1_150000-199999.pkl
../data/graphs_v1/graphs_v1_200000-249999.pkl
../data/graphs_v1/graphs_v1_250000-299999.pkl
../data/graphs_v1/graphs_v1_300000-349999.pkl
../data/graphs_v1/graphs_v1_350000-399999.pkl
../data/graphs_v1/graphs_v1_400000-449999.pkl
../data/graphs_v1/graphs_v1_450000-499999.pkl
../data/graphs_v1/graphs_v1_500000-549999.pkl
../data/graphs_v1/graphs_v1_550000-599999.pkl
../data/graphs_v1/graphs_v1_600000-649999.pkl
../data/graphs_v1/graphs_v1_650000-699999.pkl
../data/graphs_v1/graphs_v1_700000-749999.pkl
../data/graphs_v1/graphs_v1_750000-799999.pkl
../data/graphs_v1/graphs_v1_800000-849999.pkl
../data/graphs_v1/graphs_v1_850000-899999.pkl
../data/graphs_v1/graphs_v1_900000-949999.pkl
../data/graphs_v1/graphs_v1_950000-999999.pkl
../data/graphs_v1/graphs_v1_1000000-1049999.pkl
../data/graphs_v1/graphs_v1_1050000-1099

### Filtering and scaling

In [4]:
# Filter graph dataset
filter_values = df_filter['filter'].values

# DotaV1 data handling (two graphs for every match: 0-49999 radiant, 0-49999 dire, 50000-99999 radiant, etc.)
step = 50000
filter_values_V1 = []
for i in range(0,int(np.ceil(len(filter_values)/step))):
    start = i*step
    end = start+step
    # Add filters for match range twice
    filter_values_V1 = np.append(filter_values_V1, filter_values[start:end])
    filter_values_V1 = np.append(filter_values_V1, filter_values[start:end])

# Get indices of True values in filters
filter_idx = [i for i, x in enumerate(filter_values_V1) if x] # DotaV1
# filter_idx = [i for i, x in enumerate(filter_values) if x] # DotaV2

graphs_filt = graphs[filter_idx]

In [5]:
# MinMax Scaler model to normalise features from 0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df_features.iloc[:,3:].to_numpy())

MinMaxScaler()

In [6]:
# Loop through each graph and scale feature matrix and drop attack_backswing feature
print('Scaling graph dataset feature matrices:')
for i in range(0,len(graphs_filt)):
    if(i%100000==0):
        print(i)
    graphs_filt[i].x = scaler.transform(graphs_filt[i].x) # scale feature matrix
    graphs_filt[i].x = np.delete(graphs_filt[i].x, 14, 1) # remove attack_backswing as a feature

Scaling graph dataset feature matrices:
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000


In [78]:
from spektral.data import BatchLoader
from spektral.layers import GATConv, GCNConv, GlobalAvgPool, GlobalMaxPool, GlobalSumPool, GlobalAttentionPool
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
# from tensorflow.config import list_physical_devices
import random

In [8]:
# print("Num GPUs Available: ", len(list_physical_devices('GPU')))

In [81]:
# Train/valid/test split
d = graphs_filt[0:20000] # Graph data

np.random.seed(10)
idxs = np.random.permutation(len(d))
split_va, split_te = int(0.64 * len(d)), int(0.8 * len(d)) #64% training, 16% validation, 20% test
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
data_tr = d[idx_tr]
data_va = d[idx_va]
data_te = d[idx_te]

### Model 1.0 - Baseline

In [83]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_0(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(19, activation='relu')
        self.flatten = Flatten()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.flatten(x)
        # x.shape is [n_batches, 5*channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_0 = Net_1_0()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_0.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_0 = model_1_0.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

Epoch 1/50


c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\spektral\data\utils.py:221: UserWarning:

you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.



50/50 [==============================] - 1s 10ms/step - loss: 0.6948 - binary_accuracy: 0.5082 - val_loss: 0.6931 - val_binary_accuracy: 0.4972
Epoch 2/50
50/50 [==============================] - 0s 8ms/step - loss: 0.6911 - binary_accuracy: 0.5269 - val_loss: 0.6876 - val_binary_accuracy: 0.5566
Epoch 3/50
50/50 [==============================] - 0s 8ms/step - loss: 0.6890 - binary_accuracy: 0.5374 - val_loss: 0.6880 - val_binary_accuracy: 0.5281
Epoch 4/50
50/50 [==============================] - 0s 8ms/step - loss: 0.6886 - binary_accuracy: 0.5394 - val_loss: 0.6898 - val_binary_accuracy: 0.5362
Epoch 5/50
50/50 [==============================] - 0s 8ms/step - loss: 0.6882 - binary_accuracy: 0.5394 - val_loss: 0.6838 - val_binary_accuracy: 0.5550
Epoch 6/50
50/50 [==============================] - 0s 8ms/step - loss: 0.6875 - binary_accuracy: 0.5367 - val_loss: 0.6839 - val_binary_accuracy: 0.5584
Epoch 7/50
50/50 [==============================] - 0s 8ms/step - loss: 0.6860 - binar

In [84]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_0.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_0.history['binary_accuracy']
validation_accuracy = fit_log_1_0.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_0_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_0_accuracy.csv', index=False)

### Model 1.1 - GATConv

In [85]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_1(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(19, activation='relu')
        self.flatten = Flatten()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x =self.flatten(x)
        # x.shape is [n_batches, 5*channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_1 = Net_1_1()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_1.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_1 = model_1_1.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

Epoch 1/50
50/50 [==============================] - 1s 15ms/step - loss: 0.6933 - binary_accuracy: 0.5113 - val_loss: 0.6911 - val_binary_accuracy: 0.5322
Epoch 2/50
50/50 [==============================] - 1s 10ms/step - loss: 0.6927 - binary_accuracy: 0.5086 - val_loss: 0.6901 - val_binary_accuracy: 0.5362
Epoch 3/50
50/50 [==============================] - 0s 10ms/step - loss: 0.6915 - binary_accuracy: 0.5198 - val_loss: 0.6899 - val_binary_accuracy: 0.5472
Epoch 4/50
50/50 [==============================] - 0s 10ms/step - loss: 0.6908 - binary_accuracy: 0.5207 - val_loss: 0.6898 - val_binary_accuracy: 0.5519
Epoch 5/50
50/50 [==============================] - 1s 10ms/step - loss: 0.6902 - binary_accuracy: 0.5301 - val_loss: 0.6898 - val_binary_accuracy: 0.5459
Epoch 6/50
50/50 [==============================] - 0s 10ms/step - loss: 0.6891 - binary_accuracy: 0.5299 - val_loss: 0.6863 - val_binary_accuracy: 0.5519
Epoch 7/50
50/50 [==============================] - 1s 10ms/step - los

In [86]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_1.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_1.history['binary_accuracy']
validation_accuracy = fit_log_1_1.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_1_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_1_accuracy.csv', index=False)

### Model 1.2 Average Pooling

In [87]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_2(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(19, activation='relu')
        self.pool1 = GlobalAvgPool()
        self.flatten = Flatten()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_2 = Net_1_2()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_2.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_2 = model_1_2.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

Epoch 1/50
50/50 [==============================] - 1s 9ms/step - loss: 0.7221 - binary_accuracy: 0.5039 - val_loss: 0.6952 - val_binary_accuracy: 0.5006
Epoch 2/50
50/50 [==============================] - 0s 7ms/step - loss: 0.6948 - binary_accuracy: 0.5054 - val_loss: 0.6932 - val_binary_accuracy: 0.5159
Epoch 3/50
50/50 [==============================] - 0s 7ms/step - loss: 0.6936 - binary_accuracy: 0.5138 - val_loss: 0.6929 - val_binary_accuracy: 0.5116
Epoch 4/50
50/50 [==============================] - 0s 7ms/step - loss: 0.6924 - binary_accuracy: 0.5203 - val_loss: 0.6906 - val_binary_accuracy: 0.5347
Epoch 5/50
50/50 [==============================] - 0s 6ms/step - loss: 0.6915 - binary_accuracy: 0.5300 - val_loss: 0.6910 - val_binary_accuracy: 0.5284
Epoch 6/50
50/50 [==============================] - 0s 7ms/step - loss: 0.6908 - binary_accuracy: 0.5276 - val_loss: 0.6882 - val_binary_accuracy: 0.5344
Epoch 7/50
50/50 [==============================] - 0s 6ms/step - loss: 0.68

In [88]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_2.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_2.history['binary_accuracy']
validation_accuracy = fit_log_1_2.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_2_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_2_accuracy.csv', index=False)

### Model 1.3 - Max Pooling

In [89]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_3(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(19, activation='relu')
        self.pool1 = GlobalMaxPool()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_3 = Net_1_3()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_3.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_3 = model_1_3.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

Epoch 1/50
50/50 [==============================] - 1s 9ms/step - loss: 0.6959 - binary_accuracy: 0.5005 - val_loss: 0.6911 - val_binary_accuracy: 0.5284
Epoch 2/50
50/50 [==============================] - 0s 6ms/step - loss: 0.6930 - binary_accuracy: 0.5126 - val_loss: 0.6950 - val_binary_accuracy: 0.4981
Epoch 3/50
50/50 [==============================] - 0s 6ms/step - loss: 0.6923 - binary_accuracy: 0.5135 - val_loss: 0.6883 - val_binary_accuracy: 0.5491
Epoch 4/50
50/50 [==============================] - 0s 7ms/step - loss: 0.6915 - binary_accuracy: 0.5187 - val_loss: 0.6904 - val_binary_accuracy: 0.5372
Epoch 5/50
50/50 [==============================] - 0s 6ms/step - loss: 0.6909 - binary_accuracy: 0.5220 - val_loss: 0.6890 - val_binary_accuracy: 0.5309
Epoch 6/50
50/50 [==============================] - 0s 7ms/step - loss: 0.6905 - binary_accuracy: 0.5275 - val_loss: 0.6874 - val_binary_accuracy: 0.5422
Epoch 7/50
50/50 [==============================] - 0s 7ms/step - loss: 0.69

In [90]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_3.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_3.history['binary_accuracy']
validation_accuracy = fit_log_1_3.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_3_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_3_accuracy.csv', index=False)

### Model 1.4 - Sum Pooling

In [91]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_4(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(19, activation='relu')
        self.pool1 = GlobalSumPool()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_4 = Net_1_4()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_4.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_4 = model_1_4.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

Epoch 1/50
50/50 [==============================] - 1s 9ms/step - loss: 0.8425 - binary_accuracy: 0.5119 - val_loss: 0.7831 - val_binary_accuracy: 0.5362
Epoch 2/50
50/50 [==============================] - 0s 6ms/step - loss: 0.7689 - binary_accuracy: 0.5188 - val_loss: 0.7478 - val_binary_accuracy: 0.5259
Epoch 3/50
50/50 [==============================] - 0s 6ms/step - loss: 0.7413 - binary_accuracy: 0.5275 - val_loss: 0.7224 - val_binary_accuracy: 0.5356
Epoch 4/50
50/50 [==============================] - 0s 6ms/step - loss: 0.7283 - binary_accuracy: 0.5261 - val_loss: 0.7172 - val_binary_accuracy: 0.5403
Epoch 5/50
50/50 [==============================] - 0s 6ms/step - loss: 0.7185 - binary_accuracy: 0.5250 - val_loss: 0.7063 - val_binary_accuracy: 0.5416
Epoch 6/50
50/50 [==============================] - 0s 6ms/step - loss: 0.7089 - binary_accuracy: 0.5317 - val_loss: 0.7009 - val_binary_accuracy: 0.5344
Epoch 7/50
50/50 [==============================] - 0s 6ms/step - loss: 0.70

In [92]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_4.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_4.history['binary_accuracy']
validation_accuracy = fit_log_1_4.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_4_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_4_accuracy.csv', index=False)

### Model 1.5 - Attention Pooling

In [93]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_5(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(19, activation='relu')
        self.pool1 = GlobalAttentionPool(19)
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_5 = Net_1_5()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_5.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_5 = model_1_5.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

Epoch 1/50
50/50 [==============================] - 1s 12ms/step - loss: 1.1101 - binary_accuracy: 0.5026 - val_loss: 0.7248 - val_binary_accuracy: 0.5400
Epoch 2/50
50/50 [==============================] - 0s 8ms/step - loss: 0.7369 - binary_accuracy: 0.5118 - val_loss: 0.7120 - val_binary_accuracy: 0.5419
Epoch 3/50
50/50 [==============================] - 0s 8ms/step - loss: 0.7225 - binary_accuracy: 0.5149 - val_loss: 0.7009 - val_binary_accuracy: 0.5406
Epoch 4/50
50/50 [==============================] - 0s 8ms/step - loss: 0.7127 - binary_accuracy: 0.5141 - val_loss: 0.7036 - val_binary_accuracy: 0.5166
Epoch 5/50
50/50 [==============================] - 0s 8ms/step - loss: 0.7068 - binary_accuracy: 0.5114 - val_loss: 0.6943 - val_binary_accuracy: 0.5331
Epoch 6/50
50/50 [==============================] - 0s 8ms/step - loss: 0.7029 - binary_accuracy: 0.5127 - val_loss: 0.6970 - val_binary_accuracy: 0.5191
Epoch 7/50
50/50 [==============================] - 0s 9ms/step - loss: 0.7

In [94]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_5.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_5.history['binary_accuracy']
validation_accuracy = fit_log_1_5.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_5_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_5_accuracy.csv', index=False)

### Model 1.x - Reduced MMR

In [ ]:
# Filter graph dataset
filter_values = df_filter['filter'].values

# DotaV1 data handling (two graphs for every match: 0-49999 radiant, 0-49999 dire, 50000-99999 radiant, etc.)
step = 50000
filter_values_V1 = []
for i in range(0,int(np.ceil(len(filter_values)/step))):
    start = i*step
    end = start+step
    # Add filters for match range twice
    filter_values_V1 = np.append(filter_values_V1, filter_values[start:end])
    filter_values_V1 = np.append(filter_values_V1, filter_values[start:end])

# Get indices of True values in filters
filter_idx = [i for i, x in enumerate(filter_values_V1) if x] # DotaV1
# filter_idx = [i for i, x in enumerate(filter_values) if x] # DotaV2

# graphs_filt = graphs[filter_idx]

In [37]:
filter_values

array([ True,  True, False, ...,  True,  True,  True])

### Extra useful code

In [117]:
loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print(f"Test accuracy: {np.round(loss[1],4)*100}%")

32/32 [==============================] - 0s 1ms/step - loss: nan - binary_accuracy: 0.5090
Test accuracy: 50.9%


c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


In [141]:
loader_single = BatchLoader(data_va, batch_size=128)
loss = model.evaluate(loader_single.load(), steps=loader_single.steps_per_epoch)
print(f"Validation accuracy: {np.round(loss[1],4)*100}%")

50/50 [==============================] - 0s 1ms/step - loss: nan - binary_accuracy: 0.5800
Test accuracy: 57.99999999999999%


c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)
